In [1]:
import os
from IPython.display import Markdown, display
import logging
import sys
from sqlalchemy.engine import URL
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
    text
)
from llama_index.core.query_engine import NLSQLTableQueryEngine
from llama_index.core import SQLDatabase
from llama_index.core.retrievers import NLSQLRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
import pandas as pd
from http.client import HTTPConnection
from sqlalchemy import MetaData, inspect
import re


In [12]:
def send_message_streaming(message):
    connection = HTTPConnection('host.docker.internal', 8000)
    connection.putrequest('POST', '/', skip_host=True)
    connection.putheader('Content-Type', 'text/plain')
    connection._http_vsn = 11
    connection._http_vsn_str = 'HTTP/1.1'
    encoded_message = message.encode('utf-8')
    connection.putheader('Content-Length', str(len(encoded_message)))
    connection.endheaders()
    connection.send(encoded_message)
    response = connection.getresponse()

    if response.status == 200:
        while True:
            chunk = response.readline().decode('utf-8')
            if not chunk:
                return
            yield chunk.strip()
    else:
        raise Exception(f"Error: Server responded with status {response.status}")

def send_message(message):
    response = ""
    for data in send_message_streaming(message):
        response += data + '\n'
    return response

def get_db_metadata(schema=None):
    schema = schema or 'dbo'
    metadata_obj = MetaData(schema=schema)
    metadata_obj.reflect(bind=engine)

    tables_info = {}
    for table_name, table in metadata_obj.tables.items():
        columns = []
        for column in table.columns:
            columns.append({
                "name": column.name,
                #"type": str(column.type).replace(' COLLATE "SQL_Latin1_General_CP1_CI_AS',''),
                #"primary_key": column.primary_key,
            })
        tables_info[table_name] = columns

    return tables_info

# def get_db_metadata(schema=None):
#     metadata_obj = MetaData(schema=schema)
#     metadata_obj.reflect(bind=engine)
#     schema = schema or 'dbo'
#     inspector = inspect(engine)
    
#     db_metadata = {
#         "schema": schema,
#         "tables": []
#     }
#     for table_name, table in metadata_obj.tables.items():
#         table_name = table_name.replace(schema+'.','')
#         table_info = {
#             "table_name": table_name,
#             #"description": inspector.get_table_comment(table_name, schema=schema).get('text', None),
#             "columns": [],
#             "primary_keys": [key.name for key in table.primary_key.columns] if table.primary_key else [],
#             #"foreign_keys": [],
#             #"indexes": [],
#         }
        
#         # Process columns
#         for column in table.columns:
#             column_info = {
#                 "name": column.name,
#                 "type": str(column.type).replace(' COLLATE "SQL_Latin1_General_CP1_CI_AS',''),
#                 #"nullable": column.nullable,
#                 #"default": str(column.default) if column.default else None,
#                 "primary_key": column.primary_key,
#             }
#             table_info["columns"].append(column_info)
        
#         # # Process foreign keys
#         # for fk in table.foreign_keys:
#         #     table_info["foreign_keys"].append({
#         #         "column": fk.parent.name,
#         #         "referenced_table": fk.column.table.name,
#         #         "referenced_column": fk.column.name
#         #     })
        
#         # # Process indexes
#         # indexes = inspector.get_indexes(table_name, schema=schema)
#         # for index in indexes:
#         #     table_info["indexes"].append({
#         #         "name": index['name'],
#         #         "columns": index['column_names'],
#         #         "unique": index.get('unique', False)
#         #     })
        
#         db_metadata["tables"].append(table_info)
    
#     return db_metadata


def extract_query(sql_string):
    """
    Extracts the first SELECT statement from a given SQL string until the first semicolon `;`.

    Args:
        sql_string (str): The SQL string.

    Returns:
        str: The extracted SELECT statement.
    """
    # Find the index of the first occurrence of `SELECT`
    select_index = sql_string.find('SELECT')
    
    # If no SELECT found, return empty string
    if select_index == -1:
        return ''
    
    # Slice the string from the first SELECT
    sql_string = sql_string[select_index:]

    # Replace `LIMIT` with an empty string or remove it (optional, based on your use case)
    sql_string = sql_string.replace('LIMIT', ';')
    sql_string = sql_string.replace('`', ';')

    # Find the first semicolon that marks the end of the query
    sc_index = sql_string.find(';')

    # If no semicolon is found, the query continues till the end of the string
    if sc_index == -1:
        sc_index = len(sql_string)

    # Extract the query up to the semicolon and replace newlines with spaces
    select_query = sql_string[:sc_index].replace('\n', ' ')

    return select_query.strip()

# def extract_query(sql_string):
#     """
#     Extracts the first SELECT statement from a given SQL string until the first semicolon `;`.

#     Args:
#         sql_string (str): The SQL string.

#     Returns:
#         str: The extracted SELECT statement.
#     """
#     # Find the index of the first occurrence of `SELECT`
#     select_index = sql_string.find('SELECT')
    
#     # If no SELECT found, return empty string
#     if select_index == -1:
#         return ''
    
#     select_query = re.search(r'```(.*?)```', sql_string, re.DOTALL)
#     select_query = select_query.group(1).replace('sql','').strip()
#     return select_query

def query(response):
    with engine.connect() as connection:
        response = response.replace('SELECT','SELECT TOP 10') if 'SELECT TOP'  not in response else response
        result = connection.execute(text(f'{response}'))

        # Initialize an empty list to store the rows
        rows_list = []

        # Fetch all rows and append them to the list
        for row in result.fetchall():
            rows_list.append(row)

    # The connection will be automatically closed after exiting the 'with' block

    # Print the list of rows
    return(rows_list)

def extract_query_error(error_str):
    # Regular expression to extract the error message
    match = re.search(r"Invalid column name '.*?'\.", error_str)
    match = re.search(r"Ambiguous column name '.*?'\.", error_str)
    if match:
        extracted_error = match.group(0)
        return extracted_error
    else:
        return f"No match found for error {error_str}"
    
def query_error_retry(query, sql_exception):
    modified_tables = [key.replace('SalesLT.', '') for key in db_metadata.keys()]
    used_tables = [table for table in re.findall(r'\b(?:' + '|'.join(re.escape(table) for table in modified_tables) + r')\b', query)]
    filtered_db_metadata = {key: value for key, value in db_metadata.items() if key.replace('SalesLT.', '') in used_tables}
    sql_exception=extract_query_error(sql_exception)
    
    if "Ambiguous column name" in sql_exception:
        
        query_str= f'A query: {query}.  falhou com o seguinte erro: {sql_exception}.check for all tables and columns. review the query, make sure all table names and columns are right'
        
    else:
        
        query_str= f'A query: {query}.  falhou com o seguinte erro: {sql_exception}. generate working query for schema {filtered_db_metadata}. review the query, make sure all table names and columns are right'
        
    return query_str


In [3]:
# logging.basicConfig(stream=sys.stdout, level=logging.INFO)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

server = 'host.docker.internal'
database = 'AdventureWorksLT2022'
username = 'sa'
password = 'Zantut123@'
driver = '{ODBC Driver 18 for SQL Server}'

# Connect to the SQL Server database
conn_str = f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password};TrustServerCertificate=yes'
connection_url = URL.create(
    "mssql+pyodbc", 
    query={"odbc_connect": conn_str}
)
engine = create_engine(connection_url)
db_metadata = get_db_metadata(schema='SalesLT')
response = send_message("Ola! voce sabe sql poderia me ajudar com umas queries?")
print("Response :", response)

Response : Sim, eu posso ajudar com queries SQL. Por favor, dize-me o que precisas.



In [4]:
query_prefix_1='Dado o banco de dados: \n '
query_prefix_2='\n Forneça uma query MSSQL para a seguinte pergunta: '
query_prefix_3='Dada a pergunta: '
query_prefix_4=' e com a seguinte resposta: \n '
query_suffix_1= '\ntermine a query com ponto e virgula.'
query_suffix_2='estruture uma resposta com os dados. forneca apenas o texto, sem nada a mais'

In [13]:
input_query = "quais clientes mais compraram?"
query_str = input_query
response = send_message(query_prefix_1+str(db_metadata).replace('FacadeDict','')+query_prefix_2+query_str+'make sure all table names and columns are right')
#print(response)
query_str = extract_query(response)
#print(query_str+'\n')
try:
    query_result = str(query(query_str))
    print('O resultado é :\n'+query_result)
    print('A query utilizada foi a seguinte:\n',query_str)
except Exception as e:
    print('********refinando query********')
    response=send_message(query_error_retry(query_str, str(e)))
    query_str = extract_query(response)
    query_result = str(query(query_str))
    print('O resultado é :\n'+query_result)
    print('A query utilizada foi a seguinte:\n',query_str)

********refinando query********


DBAPIError: (pyodbc.Error) ('HY090', '[HY090] [unixODBC][Driver Manager]Invalid string or buffer length (0) (SQLExecDirectW)')
(Background on this error at: https://sqlalche.me/e/20/dbapi)

In [7]:
db_metadata

{'SalesLT.Address': [{'name': 'AddressID'},
  {'name': 'AddressLine1'},
  {'name': 'AddressLine2'},
  {'name': 'City'},
  {'name': 'StateProvince'},
  {'name': 'CountryRegion'},
  {'name': 'PostalCode'},
  {'name': 'rowguid'},
  {'name': 'ModifiedDate'}],
 'SalesLT.Customer': [{'name': 'CustomerID'},
  {'name': 'NameStyle'},
  {'name': 'Title'},
  {'name': 'FirstName'},
  {'name': 'MiddleName'},
  {'name': 'LastName'},
  {'name': 'Suffix'},
  {'name': 'CompanyName'},
  {'name': 'SalesPerson'},
  {'name': 'EmailAddress'},
  {'name': 'Phone'},
  {'name': 'PasswordHash'},
  {'name': 'PasswordSalt'},
  {'name': 'rowguid'},
  {'name': 'ModifiedDate'}],
 'SalesLT.CustomerAddress': [{'name': 'CustomerID'},
  {'name': 'AddressID'},
  {'name': 'AddressType'},
  {'name': 'rowguid'},
  {'name': 'ModifiedDate'}],
 'SalesLT.Product': [{'name': 'ProductID'},
  {'name': 'Name'},
  {'name': 'ProductNumber'},
  {'name': 'Color'},
  {'name': 'StandardCost'},
  {'name': 'ListPrice'},
  {'name': 'Size'},
